A.S. Lundervold, 16.01.2020.

# Introduksjon

Som diskutert i introduksjonen til Lab 3 er *virtual screening* og analyse av molekylers *affinitet* til et target, basert på deres molekylære beskrivelser (fingerprint), en sentral del av **drug discovery**. 

Dette kalles ofte **QSAR**: Quantitative Structure-Activity Relationship.

Vi ser på et eksempel.


# Setup

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from pathlib import Path

# Data

Vi bruker et (utvalg av) et datasett fra en konkurranse på Kaggle organisert av Merck i 2013. (Om du er interessert kan du laste ned hele datasettet fra https://www.kaggle.com/c/MerckActivity.) 

> Help enable the development of safe, effective medicines.

> When developing new medicines it is important to identify molecules that are highly active toward their intended targets but not toward other targets that might cause side effects. The objective of this competition is to identify the best statistical techniques for predicting biological activities of different molecules, both on- and off-target, given numerical descriptors generated from their chemical structures.

> The challenge is based on 15 molecular activity data sets, each for a biologically relevant target. Each row corresponds to a molecule and **contains descriptors derived from that molecule's chemical structure** ["fingerprints"].

Vårt datasett består av et av de femten settene fra MerckActivity. 

> Merk at dette er et **regresjonsproblem**: vi ønsker ikke å predikere en *klasse*, slik vi har gjort tidligere, men en **kontinuerlig verdi**. Vi må derfor forholde oss til litt andre modeller, og spesielt også andre teknikker for å evaluere våre resultater. 

# Last inn og se på data

Vi har plassert datasettet i katalogen `../data/drug`:

In [ ]:
DATA = Path('../data/drug')

In [ ]:
act = pd.read_csv(DATA/"ACT15_competition_training.csv")

In [ ]:
act.info()

Hver rad tilhører ett molekyl, og består av 5552 features (descriptors / fingerprint) av molekylet samt dets aktivitet med hensyn på target. Se `ELMED219-Lab3-Drug_discovery-RDKit.ipynb` for en gjennomgang av hvordan slike fingerprints kan genereres.

In [ ]:
act.head()

Som vanlig lager vi en X (features / fingerprints) og y (det vi ønsker å predikere):

In [ ]:
X = act.drop(["MOLECULE", "Act"], axis=1)
y = act['Act']

## Plots

Hva er fordelingen til aktivitetsmålene?

In [ ]:
y.hist()
plt.show()

Det er interessant å se på fordelingen av verdiene i de ulike søylene. Siden det er såpass mange velger vi oss ut et tilfeldig utvalg for plotting:

In [ ]:
import random

In [ ]:
# Hver gang denne cellen kjøres plukkes det ut 9 features tilfeldig.
# Du må gjerne forsøke flere ganger

features = random.choices(X.columns, k=9)
fig, ax = plt.subplots(figsize=(12,12))
X[features].hist(ax=ax)
plt.show()

Det kan se ut som at deskriptorene stort sett er 0 for alle molekyler, med relativt få unntak. Dette er normalt for fingerprinting.  

# Feature engineering

I denne oppgaven er det mye å hente på å studere features nærmere. Vi skal ikke gå dypt inn i dette, bortsett fra at vi dropper features som har samme eller nesten samme verdi for alle molekyler, og dermed har liten forklaringsverdi.

Vi kan fjerne alle features der den nest mest vanlige verdien er svært sjelden:

In [ ]:
def get_two_most_frequent(X, feature):
    """
    Function to grab the two most frequent values for the feature.
    Returns 0 for the second most common if the feature is constant.
    """
    
    try:
        first = X[feature].value_counts().iloc[0]
        second = X[feature].value_counts().iloc[1]
        return first, second
    except:
        return first, 0

In [ ]:
# Vi kan selv velge threshold:
threshold = 30

# Vi velger en faktor slik at features droppes dersom den mest vanlige 
# verdien er såpass mange ganger høyere enn threshold

factor = 10

to_drop = []

for feature in X.columns:
    first, second = get_two_most_frequent(X, feature)
    if (first > factor*threshold) and (second < threshold):
        to_drop.append(feature)


In [ ]:
len(to_drop), to_drop[:10]

Vi dropper så alle disse:

In [ ]:
X = X.drop(columns=to_drop)

Nå er datasettet noe mer håndterlig:

In [ ]:
X.info()

# Train-test-split

> NB: Erfaringen til deltakerne i Kaggle-konkurransen var at hvordan en splittet ut sitt valideringssett hadde stor innflytelse på hvor godt ytelsen på dette samsvarte med ytelsen på test-settet til Kaggle. 

Vi velger her å overse dette og bruke en enkel, random split av data, der 25% brukes som test-data:

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Modell

Som vanlig bruker vi en random forest, denne gangen for regresjon:

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

In [ ]:
rf.fit(X_train, y_train)

Vår modell er trent og vi kan finne dens prediksjoner på testdata:

In [ ]:
y_pred = rf.predict(X_test)

# Evaluering

Her er de første 10 fasitsvar og tilhørende prediksjoner:

In [ ]:
list(zip(y_test, y_pred))[:10]

Med `.score` på en random forest regressor beregnes såkalt $R^2$ score. Se https://en.wikipedia.org/wiki/Coefficient_of_determination. Den beste mulige scoren er 1.0.

In [ ]:
rf.score(X_test, y_test)

Vi kan velge andre mål for å evaluere regressoren. Her er to mye brukte mål i regresjon:

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
mean_squared_error(y_test, y_pred)

## Plots

Det er også nyttig å plotte prediksjonene versus de korrekte verdiene i et scatter plot. 

Her er en funksjon for å oppnå dette, som også legger til en regresjonslinje:

In [ ]:
import seaborn as sns
from sklearn.metrics import r2_score
def evaluate(y_test, y_pred):
    print(f"R2 score er: {r2_score(y_test, y_pred).round(3)}")
    print(f"Mean absolute error er: {mean_absolute_error(y_test, y_pred).round(3)}")
    print(f"Mean squared error er: {mean_squared_error(y_test, y_pred).round(3)}")
    
    plt.figure(figsize=(12,8))
    sns.regplot(x=y_test, y=y_pred, line_kws={"color":"g","lw":3}, ci=0)
    plt.show()  

In [ ]:
evaluate(y_test, y_pred)

Vi ser visuelt at modellen har plukket opp noen sammenhenger og er i stand til å stort sett predikere rimelige verdier for `Activity`.

# Fintuning av modellen

Som diskutert tidligere kan vi fin-tune modeller ved å søke etter gode hyperparametre. La oss forsøke.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)

In [ ]:
param_grid = {
    'max_depth': [7, None],
    'min_samples_leaf': [1, 2],
    'n_estimators': [100, 500]
}

In [ ]:
gs = GridSearchCV(rf, param_grid=param_grid, cv=3, n_jobs=-1)

In [ ]:
# OBS: Ikke kjør koden i denne cellen med mindre du har god tid. 
# Se nedenfor for en snarvei.

#gs.fit(X_train, y_train)

# Her er den beste modellen vi fant i søket:
#model = gs.best_estimator_

In [ ]:
#model

**OBS:** Kjør følgende kode istedenfor cellene over for å spare tid. Her laster vi inn den beste modellen funnet via søket.

In [ ]:
import pickle
#pickle.dump(model, open('grid_search-drug-model', 'wb'))
model = pickle.load(open('grid_search-drug-model', 'rb'))

Vi ser at parametrene til denne er litt forskjellige fra de til modellen vi brukte tidligere:

In [ ]:
model

Hvor god er denne modellen?

In [ ]:
y_pred = model.predict(X_test)

evaluate(y_test, y_pred)

In [ ]:
model.score(X_test, y_test)

Modellen er altså ørlitte grann bedre enn vår første. 

# Feature importance

Som vanlig er det interessant å se på _feature importance_

In [ ]:
importances = model.feature_importances_

# Finn indeks til features med høyest importance
# sortert fra størst til minst: 
indices = np.argsort(importances)[::-1]

for f in range(10): 
    print(f'{X.columns[indices[f]]}: {np.round(importances[indices[f]],2)}')

# Permutation importance

...og, siden vi vet at feature importance er ustabilt, også på _permutation importance_.

> På grunn av det store antall features vil dette ta *veldig* lang tid. Jeg har derfor kommentert bort selve kjøringen av permutation importance nedenfor.. Se output i bildet nederst. 

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(model, random_state=42).fit(X_test, y_test)

In [ ]:
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

In [ ]:
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Her er resultatet:

<img width=35% src="assets/perm_importance_drug_gridsearch.png">

Det er ikke vanskelig å tenke seg at denne informasjonen er veldig verdifull, gitt at en har en god modell og en vet hva "D_7656" og "D_5790" sier om den kjemiske forbindelsen.

# Ekstra: PCA

En nyttig teknikk når en arbeider med et datasett med et stort antall features er å bruke _dimensjonalitetsreduksjons-teknikker_. Én slik er såkalt _prinicipal component analysis_ (PCA).

En ulempe med PCA er at vi har mister den direkte koblingen til orginalfeatures. Dersom en er interessert i å _forstå_ modellens prediksjoner er dette en stor ulempe. Men om vi kun er interessert i prediksjonsytelse kan dette være en fordel: med færre features kan vi f.eks. søke etter bedre hyperparametre. 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
n_components = 100
#n_components = 'mle' # 'mle' gir oss et automatisk estimat på hvor mange features som behøves

In [ ]:
pca = PCA(n_components=n_components) 
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

Nå har vi mye færre features å jobbe med:

In [ ]:
# Før
X_train.shape

In [ ]:
# Nå
X_train_pca.shape

Vi kan så søke gjennom et større grid for å finne gode hyperparametre for en random forest:

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, random_state=42)

In [ ]:
param_grid = {
    'max_depth': [7, 50, None],
    'min_samples_leaf': [1, 2],
    'n_estimators': [300, 500, 600]
}

In [ ]:
gs_pca = GridSearchCV(rf, param_grid=param_grid, cv=3)

In [ ]:
gs_pca.fit(X_train_pca, y_train)

# Her er den beste modellen vi fant i søket:
model_pca = gs_pca.best_estimator_

In [ ]:
model_pca

Hvor godt gjør den det? 

In [ ]:
y_pred_pca = model_pca.predict(X_test_pca)

evaluate(y_test, y_pred_pca)

På høyde med modellen basert på mye flere features!